# PCA PLAYGROUND

In [46]:
import pandas as pd
df_annot = pd.read_csv('/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/annot_new.csv')

In [47]:
import numpy as np
import pandas as pd
import plotly_express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotly.express.colors import sample_colorscale
from sklearn.preprocessing import minmax_scale
from sklearn import preprocessing



df_annot_final = pd.read_csv(
    '/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/annot_new2.csv')

In [48]:
GENERIC_FEATURES = ['min_f', 'max_f', 'centroid_f',
                    'duration_t', 'bandwidth_f', 'area_tf']

SHAPE_FEATURES = ['shp_002', 'shp_003', 'shp_004', 'shp_005', 'shp_006', 'shp_007',
                  'shp_008', 'shp_009', 'shp_010', 'shp_011', 'shp_012', 'shp_013',
                  'shp_014', 'shp_015', 'shp_016', 'shp_017', 'shp_018', 'shp_019',
                  'shp_020', 'shp_021', 'shp_022', 'shp_023', 'shp_024', 'shp_025',
                  'shp_026', 'shp_027', 'shp_028', 'shp_029', 'shp_030', 'shp_031',
                  'shp_032', 'shp_033', 'shp_034', 'shp_035', 'shp_036', 'shp_037',
                  'shp_038', 'shp_039', 'shp_040', 'shp_041', 'shp_042', 'shp_043',
                  'shp_044', 'shp_045', 'shp_046', 'shp_047', 'shp_048']


SPECTRAL_FEATURES = ['MEANf', 'VARf', 'SKEWf', 'KURTf', 'NBPEAKS', 'LEQf',
                     'ENRf', 'BGNf', 'SNRf', 'Hf', 'EAS', 'ECU', 'ECV', 'EPS', 'EPS_KURT', 'EPS_SKEW', 'ACI',
                     'NDSI', 'ROU']

TEMPORAL_FEATURES = ['ZCR', 'MEANt', 'VARt',
                     'SKEWt', 'KURTt', 'Ht']


In [49]:
df = df_annot_final.copy()
df = df.reset_index()

features = GENERIC_FEATURES

df = df.dropna(how='all')
# Scale Features
data_scaled = pd.DataFrame(preprocessing.scale(
    df[features]), columns=df[features].columns)
# PCA
pca = PCA(n_components=4)
components = pca.fit_transform(data_scaled)
total_var = pca.explained_variance_ratio_.sum() * 100

# Dump components relations with features:
weights = (pd.DataFrame(pca.components_,
            columns=data_scaled.columns, index=[f'PC-{i+1}' for i in range(4)]))
weights = weights.reset_index()

# figures-----------------------
labels = {
    str(i): f"PC {i+1} ({var1:.1f}%) - EV = {var2:.1f}" for i, (var1, var2) in enumerate(zip(pca.explained_variance_ratio_ * 100, pca.explained_variance_))
}

colors_ = np.linspace(0, 1, len(df[color].unique()))
discrete_colors = sample_colorscale('Rainbow', minmax_scale(colors_))

In [50]:
pd.DataFrame(components)

,0,1,2,3
0,2.801196,1.412692,-0.897027,-0.194643
1,3.214773,2.153596,-0.401168,-0.247775
2,1.271617,1.391316,-1.209887,0.025513
3,3.760102,2.606550,-1.656707,-0.269509
4,3.713450,3.906413,-2.750265,-0.114423
...,...,...,...,...
3370,-1.244953,0.695778,1.469935,-0.359523
3371,0.003178,-0.212682,4.174981,-0.283742
3372,-0.278227,-0.338082,3.529692,-0.223591
3373,-0.131388,-0.350133,3.529681,-0.189995


In [52]:
for_glenn = pd.concat([df[['species', 'genus', 'family', 'order','biotope', 'sound_id']],pd.DataFrame(components)],axis=1)

In [58]:
for_glenn.to_csv('PC_results_glenn.csv')

In [56]:
fig = px.scatter_matrix(
    components,
    height=1000,
    labels=labels,
    dimensions=range(dimensions),
    color=df[color],
    color_discrete_sequence=discrete_colors,
    title=f'Principal component analysis of ROIs ---Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)


/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/venv/lib/python3.10/site-packages/plotly/express/_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'diagonal': {'visible': False},
              'dimensions': [{'axis': {'matches': True},
                              'label': 'PC 1 (49.3%) - EV = 3.0',
                              'values': array([2.80119595, 3.21477317, 1.27161734, 3.76010244, 3.71344979, 2.6298529 ,
                                               1.88278867, 1.15620683, 2.29649787, 2.11622933, 3.20894029, 0.55205693,
                                               1.56013424, 2.94073789, 2.64044255, 1.26881709, 0.5539406 , 0.83942547,
                                               1.89619593, 3.5277461 , 2.72387868, 2.19923313, 3.68944079, 3.12220352,
                                               2.39471936, 2.58884654, 2.52326715, 2.75941298, 0.26536036, 2.86625958,
                                               0.80079412, 2.65611759, 2.29348288])},
                             {'axis': {'matches': True},
                              'label': 'PC 2 (26.1%) - EV = 1.6',
                              'values': array([1.4126921 , 2.15359634, 1.3913161 , 2.60655016, 3.90641335, 3.08998741,
                                               2.64178582, 1.87723902, 1.71203298, 1.70179327, 4.20249487, 1.39638284,
                                               2.2717658 , 1.66929848, 3.29311346, 1.76830285, 1.12183169, 1.09492083,
                                               1.73487215, 1.52927539, 3.1605027 , 0.42826344, 1.09390583, 3.3680865 ,
                                               0.38773994, 2.47572364, 0.93944323, 2.56652233, 1.61911331, 2.7745832 ,
                                               1.14549346, 2.88680982, 1.05184351])}],
              'hovertemplate': ('color=africanoides<br>%{xaxis.' ... 'itle.text}=%{y}<extra></extra>'),
              'legendgroup': 'africanoides',
              'marker': {'color': 'rgb(150, 0, 90)', 'symbol': 'circle'},
              'name': 'africanoides',
              'showlegend': True,
              'type': 'splom'},
             {'diagonal': {'visible': False},
              'dimensions': [{'axis': {'matches': True},
                              'label': 'PC 1 (49.3%) - EV = 3.0',
                              'values': array([-0.13681598, -0.06815616, -0.17097382, -0.13971908, -0.18118598,
                                               -0.17274811, -0.1572791 , -0.08896289, -0.00262433, -0.10946535,
                                               -0.02911925, -0.13337223, -0.08495507, -0.00551868, -0.12312107,
                                               -0.14997004,  0.11492072, -0.08165403, -0.02018469, -0.08495507,
                                               -0.11690168, -0.06467321])},
                             {'axis': {'matches': True},
                              'label': 'PC 2 (26.1%) - EV = 1.6',
                              'values': array([1.03442384, 1.11574051, 1.09372109, 1.14774909, 1.11463222, 1.2068179 ,
                                               1.14472615, 1.02613225, 1.30124163, 1.20491402, 1.04376954, 1.17482009,
                                               1.16641435, 1.22670567, 1.08542957, 1.23714027, 1.30143616, 1.11854571,
                                               1.29821876, 1.16641435, 1.10959201, 1.18765803])}],
              'hovertemplate': ('color=olivacea<br>%{xaxis.titl' ... 'itle.text}=%{y}<extra></extra>'),
              'legendgroup': 'olivacea',
              'marker': {'color': 'rgb(123, 0, 110)', 'symbol': 'circle'},
              'name': 'olivacea',
              'showlegend': True,
              'type': 'splom'},
             {'diagonal': {'visible': False},
              'dimensions': [{'axis': {'matches': True},
                              'label': 'PC 1 (49.3%) - EV = 3.0',
                              'values': array([0.77795315, 0.9805184 ])},
                             {'axis': {'matches': True},
                              'label': 'PC 2 (26.1%) - EV = 1.6',
                              'values': array([0.35

In [14]:
features_options='basic'
dimensions=2
color="species"

df = df_annot.copy()
df = df.reset_index()

if features_options == 'basic':
    features = GENERIC_FEATURES
if features_options == 'shapes':
    features = SHAPE_FEATURES+GENERIC_FEATURES
if features_options == 'spectral':
    features = SPECTRAL_FEATURES+GENERIC_FEATURES
if features_options == 'temporal':
    features = TEMPORAL_FEATURES+GENERIC_FEATURES
if features_options == 'all':
    features = GENERIC_FEATURES+SPECTRAL_FEATURES+TEMPORAL_FEATURES+SHAPE_FEATURES

df = df.dropna(how='all')
X = df.loc[:, features]

# normalize data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
# compute PCA, explained variance and weights
pca = PCA()
components = pca.fit_transform(X)
total_var = pca.explained_variance_ratio_.sum() * 100

indexPC = ['PC - '+str(i) for i in range(dimensions)]
df_feature_weights = pd.DataFrame(pca.components_)

# figure-----------------------
labels = {
    str(i): f"PC {i+1} ({var:.1f}%) -- sound_id = {df.loc[:,'']}"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

colors_ = np.linspace(0, 1, len(df[color].unique()))
discrete_colors = sample_colorscale('Rainbow', minmax_scale(colors_))

fig = px.scatter_matrix(
    components,
    height=1000,
    labels=labels,
    dimensions=range(dimensions),
    color=df[color],
    color_discrete_sequence=discrete_colors,
    title=f'Principal component analysis of ROIs ---Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)

fig2 = px.bar(df_feature_weights, title='Weights of features for each PC')




KeyError: ''

In [9]:
fig.show()

NameError: name 'fig' is not defined

In [5]:
labels

{'0': 'PC 1 (32.2%)',
 '1': 'PC 2 (29.9%)',
 '2': 'PC 3 (18.3%)',
 '3': 'PC 4 (10.1%)',
 '4': 'PC 5 (5.5%)',
 '5': 'PC 6 (2.2%)',
 '6': 'PC 7 (1.1%)',
 '7': 'PC 8 (0.5%)',
 '8': 'PC 9 (0.2%)',
 '9': 'PC 10 (0.0%)',
 '10': 'PC 11 (0.0%)',
 '11': 'PC 12 (0.0%)',
 '12': 'PC 13 (0.0%)',
 '13': 'PC 14 (0.0%)',
 '14': 'PC 15 (0.0%)',
 '15': 'PC 16 (0.0%)',
 '16': 'PC 17 (0.0%)',
 '17': 'PC 18 (0.0%)',
 '18': 'PC 19 (0.0%)'}

In [ ]:
from sklearn import preprocessing

data_scaled = pd.DataFrame(preprocessing.scale(df_annot[features]),columns = df_annot[features].columns) 

# PCA
pca = PCA(n_components=3)
pca.fit_transform(data_scaled)

# Dump components relations with features:
weights = (pd.DataFrame(abs(pca.components_),columns=data_scaled.columns,index = ['PC-1','PC-2','PC-3']))


In [ ]:
test = weights.reset_index()

In [ ]:
test

In [ ]:
px.bar(test,x=features, facet_col='index', barmode='group')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(cols=1, rows=dimensions,subplot_titles=([f'PC-{i+1}: abs weight of features' for i in range(dimensions)]))

for i in range(dimensions):
    fig.append_trace(go.Bar(x=features ,y = test[features].loc[i,:],text=features,textposition="inside")
    , col=1, row=i+1)
    fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
    fig.update_xaxes({'categoryorder':'total descending'})
    fig.update_xaxes(visible=False)


fig.update_layout(height=800, width=1000, showlegend=False)
fig.show()

In [ ]:
for i in range(dimensions):
    print(i)

In [ ]:
abs(weights).reset_index().groupby('index').max()

In [ ]:
weights = weights.transpose()

In [ ]:
abs(weights)

In [ ]:
px.bar(weights, barmode='group')

In [ ]:
total_var = pca.explained_variance_ratio_.sum() * 100


In [ ]:
total_var

In [ ]:
weights['index']

In [ ]:
weights = weights.reset_index()
weights['PC'] = weights['index']

In [ ]:
weights[features] = abs(weights[features])

In [ ]:
fig2 = px.bar(weights, y=features,x=px.Constant('Relative features'), facet_col = 'index', barmode='group',title='Weights of features for each PC')

In [ ]:
fig2.update_yaxes(type='category')


In [ ]:
fig2.show()

In [ ]:
weights

In [ ]:
fig3 = px.bar(weights[features].loc[2,:].sort_values())
fig3.update_layout(title = 'PC-1')


In [ ]:
fig3.show()